In [146]:

Chromatic = [0,1,2,3,4,5,6,7,8,9,10,11]
Ionian = [0,2,4,5,7,9,11]
Dorian = [0,2,3,5,7,9,10]
Phrygian = [0,1,3,5,7,8,10]
Lydian = [0,2,4,6,7,9,11]
Mixolydian = [0,2,4,5,7,9,10]
Aeolian = [0,2,3,5,7,8,10]
Locrian = [0,1,3,5,6,8,10]

#variations
Harmonic_minor = [0,2,3,5,7,8,11]
Melodic_minor = [0,2,3,5,7,9,11]

#Eastern classical modes
Bhairav = [0,1,4,5,7,8,11]
Poorvi = [0,1,4,6,7,8,11]
Marva = [0,1,4,6,7,9,11]
Todi = [0,2,3,6,7,8,11]




In [147]:
modeList = [Ionian, Dorian, Phrygian, Lydian, Mixolydian, Aeolian, Locrian, Harmonic_minor, Melodic_minor, Bhairav,Poorvi, Marva, Todi]

In [148]:
modeTuples = [ ("Ionian" , Ionian),
("Dorian" , Dorian),
("Phrygian", Phrygian),
("Lydian" , Lydian), 
("Mixolydian" , Mixolydian),
("Aeolian" , Aeolian),
("Locrian" , Locrian),
("Harmonic_minor", Harmonic_minor),
("Melodic_minor_ascend", Melodic_minor),
("Bhairav",Bhairav),
("Poorvi", Poorvi),
("Marva", Marva),
("Todi", Todi),
]

In [206]:
midinumbers = {
    'c' : 0,
    'C' : 0,
    'c#' : 1,
    'C#' : 1,
    'd-' : 1,
    'D-' : 1,
    'd' : 2,
    'D' :2,
    'd#' : 3,
    'D#' : 3,
    'e-' : 3,
    'E-' : 3,
    'e' : 4,
    'E' : 4,
    'f' : 5,
    'F' : 5,
    'f#' : 6,
    'F#' : 6,
    'g-' :6,
    'G-': 6,
    'g' : 7,
    'G' : 7,
    'g#' : 8,
    'G#' :8,
    'a-' : 8,
    'A-' :8,
    'a' : 9,
    'A' : 9,
    'a#' : 10,
    'A#' : 10,
    'b-' : 10,
    'B-' : 10,
    'b' : 11,
    'B' : 11,

}

In [149]:
from music21 import *
import scipy
from collections import Counter

In [150]:
def open_midi(midi_path, remove_drums):
    # There is an one-line method to read MIDIs
    # but to remove the drums we need to manipulate some
    # low level MIDI events.
    mf = midi.MidiFile()
    mf.open(midi_path)
    mf.read()
    mf.close()
    print ((mf.tracks))
    print(mf)
    #for eve in mf.tracks[0].events:
        #print(eve)
    if (remove_drums):
        for i in range(len(mf.tracks)):
           
            mf.tracks[i].events = [ev for ev in mf.tracks[i].events if ev.channel != 10]          
    
    
    return midi.translate.midiFileToStream(mf)

In [ ]:
class Rootnote:
    def __init__(self, noteName):
        self.noteName = noteName

   

    def __str__(self):
        return f"{self.noteName}"
    
    def asnum(self):
        return midinumbers[self.noteName]

In [ ]:
def extract_notes(midi_part):
    parent_element = []
    ret = []
    for nt in midi_part.flat.notes:        
        if isinstance(nt, note.Note):
            ret.append(max(0.0, nt.pitch.ps))
            parent_element.append(nt)
        elif isinstance(nt, chord.Chord):
            for pitch in nt.pitches:
                ret.append(max(0.0, pitch.ps))
                parent_element.append(nt)
    
    return ret, parent_element

In [186]:
def getMode(midi_filename):
    base_midi = open_midi(midi_filename, True)

    music_analysis = base_midi.analyze('key')

    rootnote = Rootnote(format(music_analysis).split(' ')[0])
    print("the root note is ", rootnote)
    a, b = extract_notes(base_midi)
    transposed = [x - rootnote.asnum() for x in a]
    refOctave = [x % 12 for x in transposed]

    freq = Counter(refOctave)
 
    freq = dict (freq) 
    sorted_notes= sorted(freq.items(), key=lambda x:x[1], reverse=True)
    mostusednotes = []
    for i,j in sorted_notes:
        mostusednotes.append(i)

    mostusednotes = mostusednotes[:7]
    mostusednotes.sort()
    print("most used notes : ", mostusednotes)
    lowest = 999
    for modeName, mode in modeTuples:
        distance = scipy.spatial.distance.hamming(mostusednotes, mode)
        if distance < lowest:
            lowest = distance 
            ourmode = modeName

    return ourmode
    



In [209]:
ourmode = getMode("AnotherBrickInTheWall.mid")


[<music21.midi.MidiTrack 0 -- 8 events>, <music21.midi.MidiTrack 1 -- 928 events>, <music21.midi.MidiTrack 2 -- 928 events>, <music21.midi.MidiTrack 3 -- 7906 events>, <music21.midi.MidiTrack 4 -- 3716 events>, <music21.midi.MidiTrack 5 -- 5584 events>, <music21.midi.MidiTrack 6 -- 320 events>, <music21.midi.MidiTrack 7 -- 320 events>, <music21.midi.MidiTrack 8 -- 120 events>, <music21.midi.MidiTrack 9 -- 6066 events>, <music21.midi.MidiTrack 10 -- 308 events>, <music21.midi.MidiTrack 11 -- 1460 events>, <music21.midi.MidiTrack 12 -- 388 events>, <music21.midi.MidiTrack 13 -- 4772 events>, <music21.midi.MidiTrack 14 -- 4774 events>, <music21.midi.MidiTrack 15 -- 6 events>]
<music21.midi.MidiFile 16 tracks>
the root note is  d
most used notes :  [0.0, 2.0, 3.0, 5.0, 7.0, 9.0, 10.0]


In [210]:
print(ourmode)

Dorian
